In [ ]:
! pip install fsspec

In [4]:
import pandas as pd
import numpy as np
import fsspec
import xml


In [40]:
df = pd.read_xml('C://Users/Raj/Downloads/mplus_topics_compressed_2023-03-02/mplus_topics_2023-03-02.xml')

In [41]:
df.head(3)

,meta-desc,title,url,id,language,date-created,also-called,full-summary,group,language-mapped-topic,mesh-heading,other-language,primary-institute,see-reference,site,related-topic
0,"If you are being tested for Type 2 diabetes, y...",A1C,https://medlineplus.gov/a1c.html,6308,English,12/22/2015,Hemoglobin A1C test,"<p>A1C is a blood test for <a href=""https://me...",Diabetes Mellitus,Prueba de hemoglobina glicosilada (HbA1c),NaN,Spanish,National Institute of Diabetes and Digestive a...,Hemoglobin A1c,NaN,None
1,Stomach aches can be painful. Find out what mi...,Abdominal Pain,https://medlineplus.gov/abdominalpain.html,3061,English,01/07/2003,Bellyache,<p>Your abdomen extends from below your chest ...,Symptoms,Dolor abdominal,NaN,Spanish,None,Stomach Ache,NaN,Pelvic Pain
2,An abortion is a medical procedure to end a pr...,Abortion,https://medlineplus.gov/abortion.html,122,English,03/19/2002,Induced Abortion,<p>An abortion is a procedure to end a pregnan...,Female Reproductive System,Aborto,NaN,Vietnamese,None,None,NaN,None


In [42]:
df.shape

(2044, 16)

In [43]:
df = df[['id', 'meta-desc', 'title', 'language','also-called', 'full-summary', 'group', 'language-mapped-topic', 'other-language', 'primary-institute', 'see-reference', 'related-topic', 'site', 'url']]
df["combined"] = (
    "Title: " + df.title.fillna('') + 
    " Meta Desc: " + df['meta-desc'].fillna('') +
    " Also Called: " + df['also-called'].fillna('') +
    " Full Summary: " + df['full-summary'].fillna('') +
    " Related Topic: " + df['related-topic'].fillna('') +
    " Group: " + df.group.fillna('') +
    " language: " + df.language.fillna('') +
    " Language Mapped Topic: " + df['language-mapped-topic'].fillna('')
)
# df = df.sample(10) # Extract sample sized random rows

In [44]:
df.shape

(2044, 15)

In [45]:
df

,id,meta-desc,title,language,also-called,full-summary,group,language-mapped-topic,other-language,primary-institute,see-reference,related-topic,site,url,combined
0,6308,"If you are being tested for Type 2 diabetes, y...",A1C,English,Hemoglobin A1C test,"<p>A1C is a blood test for <a href=""https://me...",Diabetes Mellitus,Prueba de hemoglobina glicosilada (HbA1c),Spanish,National Institute of Diabetes and Digestive a...,Hemoglobin A1c,None,NaN,https://medlineplus.gov/a1c.html,Title: A1C Meta Desc: If you are being tested ...
1,3061,Stomach aches can be painful. Find out what mi...,Abdominal Pain,English,Bellyache,<p>Your abdomen extends from below your chest ...,Symptoms,Dolor abdominal,Spanish,None,Stomach Ache,Pelvic Pain,NaN,https://medlineplus.gov/abdominalpain.html,Title: Abdominal Pain Meta Desc: Stomach aches...
2,122,An abortion is a medical procedure to end a pr...,Abortion,English,Induced Abortion,<p>An abortion is a procedure to end a pregnan...,Female Reproductive System,Aborto,Vietnamese,None,None,None,NaN,https://medlineplus.gov/abortion.html,Title: Abortion Meta Desc: An abortion is a me...
3,2238,Un aborto es un procedimiento médico para term...,Aborto,Spanish,Interrupción del embarazo,<p>Un aborto es un procedimiento para terminar...,Sistema reproductor femenino,Abortion,None,None,None,None,NaN,https://medlineplus.gov/spanish/abortion.html,Title: Aborto Meta Desc: Un aborto es un proce...
4,3064,Los abscesos se llenan de pus y pueden aparece...,Abscesos,Spanish,None,<p>Un absceso es una cavidad donde se acumula ...,Infecciones,Abscess,None,Instituto Nacional de Alergias y Enfermedades ...,None,Enfermedades infecciosas,NaN,https://medlineplus.gov/spanish/abscess.html,Title: Abscesos Meta Desc: Los abscesos se lle...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039,6604,¿Qué cantidad de ejercicio se debe hacer? Aquí...,¿Cuánto ejercicio debo hacer?,Spanish,None,<p>Hacer ejercicio en forma regular es una de ...,Aptitud física y ejercicio,How Much Exercise Do I Need?,None,None,None,None,NaN,https://medlineplus.gov/spanish/howmuchexercis...,Title: ¿Cuánto ejercicio debo hacer? Meta Desc...
2040,1907,"Si está embarazada, es probable que haya escuc...",Ácido fólico,Spanish,Ácido pteroilglutámico,"<p>El ácido fólico es una <a href=""https://med...",Alimentos y nutrición,Folic Acid,None,Oficina de Suplementos Dietéticos,None,Vitaminas,NaN,https://medlineplus.gov/spanish/folicacid.html,Title: Ácido fólico Meta Desc: Si está embaraz...
2041,1749,El ántrax o carbunco es ocasionado por una bac...,Ántrax,Spanish,Carbunco,<p>El ántrax es una enfermedad causada por el ...,Infecciones,Anthrax,None,Instituto Nacional de Alergias y Enfermedades ...,Carbunco,Biodefensa y bioterrorismo,NaN,https://medlineplus.gov/spanish/anthrax.html,Title: Ántrax Meta Desc: El ántrax o carbunco ...
2042,2108,Las úlceras por presión son áreas de piel lesi...,Úlcera por presión,Spanish,Llagas por presión,<p>Las úlceras por presión son áreas de piel l...,"Piel, cabello y uñas",Pressure Sores,None,Instituto Nacional de Artritis y Enfermedades ...,Úlcera por decúbito,None,NaN,https://medlineplus.gov/spanish/pressuresores....,Title: Úlcera por presión Meta Desc: Las úlcer...


In [46]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY") 
# print(openai.api_key)

In [47]:
# embedding model parameters
embedding_model = "text-embedding-ada-002" # second gen best model at the moment
embedding_encoding = "cl100k_base" # latest tokenizer for second gen models
max_tokens = 8000 # max tokens for second gen models and tokenizer above is 8191

In [48]:
# Remove samples that are too long
import tiktoken

encoding = tiktoken.get_encoding(embedding_encoding)
# omit topics that are too long
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens < max_tokens]
len(df)

2044

In [49]:
# Get embeddings and save them to a csv file, slow down the requests to avoid rate limit
import time
from openai.embeddings_utils import get_embedding

def get_embedding_with_delay(text):
    time.sleep(2)
    print(time.ctime())
    return get_embedding(text, engine=embedding_model)

In [50]:
# Get embeddings and save them
from openai.embeddings_utils import get_embedding
# df["embedding"] = df.combined.sample(10).apply(lambda x: get_embedding(x, engine=embedding_model))
# df["embedding"] = df.combined.apply(lambda x: get_embedding_with_delay(x))
df["embedding"] = df.combined.apply(lambda x: get_embedding_with_delay(x))
df.to_csv('mplus_10_embeddings.csv')

Thu Mar  2 17:07:51 2023
Thu Mar  2 17:07:53 2023
Thu Mar  2 17:07:55 2023
Thu Mar  2 17:07:58 2023
Thu Mar  2 17:08:00 2023
Thu Mar  2 17:08:02 2023
Thu Mar  2 17:08:04 2023
Thu Mar  2 17:08:06 2023
Thu Mar  2 17:08:08 2023
Thu Mar  2 17:08:10 2023
Thu Mar  2 17:08:12 2023
Thu Mar  2 17:08:14 2023
Thu Mar  2 17:08:17 2023
Thu Mar  2 17:08:19 2023
Thu Mar  2 17:08:21 2023
Thu Mar  2 17:08:23 2023
Thu Mar  2 17:08:25 2023
Thu Mar  2 17:08:27 2023
Thu Mar  2 17:08:30 2023
Thu Mar  2 17:08:32 2023
Thu Mar  2 17:08:34 2023
Thu Mar  2 17:08:36 2023
Thu Mar  2 17:08:38 2023
Thu Mar  2 17:08:40 2023
Thu Mar  2 17:08:42 2023
Thu Mar  2 17:08:44 2023
Thu Mar  2 17:08:47 2023
Thu Mar  2 17:08:49 2023
Thu Mar  2 17:08:52 2023
Thu Mar  2 17:08:54 2023
Thu Mar  2 17:08:56 2023
Thu Mar  2 17:08:58 2023
Thu Mar  2 17:09:00 2023
Thu Mar  2 17:09:03 2023
Thu Mar  2 17:09:05 2023
Thu Mar  2 17:09:07 2023
Thu Mar  2 17:09:09 2023
Thu Mar  2 17:09:11 2023
Thu Mar  2 17:09:13 2023
Thu Mar  2 17:09:15 2023


In [51]:
df

,id,meta-desc,title,language,also-called,full-summary,group,language-mapped-topic,other-language,primary-institute,see-reference,related-topic,site,url,combined,n_tokens,embedding
0,6308,"If you are being tested for Type 2 diabetes, y...",A1C,English,Hemoglobin A1C test,"<p>A1C is a blood test for <a href=""https://me...",Diabetes Mellitus,Prueba de hemoglobina glicosilada (HbA1c),Spanish,National Institute of Diabetes and Digestive a...,Hemoglobin A1c,None,NaN,https://medlineplus.gov/a1c.html,Title: A1C Meta Desc: If you are being tested ...,437,"[-0.004037505481392145, -0.003933656960725784,..."
1,3061,Stomach aches can be painful. Find out what mi...,Abdominal Pain,English,Bellyache,<p>Your abdomen extends from below your chest ...,Symptoms,Dolor abdominal,Spanish,None,Stomach Ache,Pelvic Pain,NaN,https://medlineplus.gov/abdominalpain.html,Title: Abdominal Pain Meta Desc: Stomach aches...,254,"[0.003938998095691204, -0.005209034308791161, ..."
2,122,An abortion is a medical procedure to end a pr...,Abortion,English,Induced Abortion,<p>An abortion is a procedure to end a pregnan...,Female Reproductive System,Aborto,Vietnamese,None,None,None,NaN,https://medlineplus.gov/abortion.html,Title: Abortion Meta Desc: An abortion is a me...,195,"[-0.022983038797974586, 0.015163933858275414, ..."
3,2238,Un aborto es un procedimiento médico para term...,Aborto,Spanish,Interrupción del embarazo,<p>Un aborto es un procedimiento para terminar...,Sistema reproductor femenino,Abortion,None,None,None,None,NaN,https://medlineplus.gov/spanish/abortion.html,Title: Aborto Meta Desc: Un aborto es un proce...,253,"[-0.031313538551330566, 0.0003964322677347809,..."
4,3064,Los abscesos se llenan de pus y pueden aparece...,Abscesos,Spanish,None,<p>Un absceso es una cavidad donde se acumula ...,Infecciones,Abscess,None,Instituto Nacional de Alergias y Enfermedades ...,None,Enfermedades infecciosas,NaN,https://medlineplus.gov/spanish/abscess.html,Title: Abscesos Meta Desc: Los abscesos se lle...,318,"[0.0016349778743460774, -0.0014277726877480745..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039,6604,¿Qué cantidad de ejercicio se debe hacer? Aquí...,¿Cuánto ejercicio debo hacer?,Spanish,None,<p>Hacer ejercicio en forma regular es una de ...,Aptitud física y ejercicio,How Much Exercise Do I Need?,None,None,None,None,NaN,https://medlineplus.gov/spanish/howmuchexercis...,Title: ¿Cuánto ejercicio debo hacer? Meta Desc...,1162,"[-0.009836418554186821, -0.0020518845412880182..."
2040,1907,"Si está embarazada, es probable que haya escuc...",Ácido fólico,Spanish,Ácido pteroilglutámico,"<p>El ácido fólico es una <a href=""https://med...",Alimentos y nutrición,Folic Acid,None,Oficina de Suplementos Dietéticos,None,Vitaminas,NaN,https://medlineplus.gov/spanish/folicacid.html,Title: Ácido fólico Meta Desc: Si está embaraz...,375,"[-0.01595480740070343, -0.013429493643343449, ..."
2041,1749,El ántrax o carbunco es ocasionado por una bac...,Ántrax,Spanish,Carbunco,<p>El ántrax es una enfermedad causada por el ...,Infecciones,Anthrax,None,Instituto Nacional de Alergias y Enfermedades ...,Carbunco,Biodefensa y bioterrorismo,NaN,https://medlineplus.gov/spanish/anthrax.html,Title: Ántrax Meta Desc: El ántrax o carbunco ...,484,"[-0.037753768265247345, -0.017405441030859947,..."
2042,2108,Las úlceras por presión son áreas de piel lesi...,Úlcera por presión,Spanish,Llagas por presión,<p>Las úlceras por presión son áreas de piel l...,"Piel, cabello y uñas",Pressure Sores,None,Instituto Nacional de Artritis y Enfermedades ...,Úlcera por decúbito,None,NaN,https://medlineplus.gov/spanish/pressuresores....,Title: Úlcera por presión Meta Desc: Las úlcer...,329,"[0.019308919087052345, 0.007406963966786861, 0..."


In [52]:
df.embedding

0       [-0.004037505481392145, -0.003933656960725784,...
1       [0.003938998095691204, -0.005209034308791161, ...
2       [-0.022983038797974586, 0.015163933858275414, ...
3       [-0.031313538551330566, 0.0003964322677347809,...
4       [0.0016349778743460774, -0.0014277726877480745...
                              ...                        
2039    [-0.009836418554186821, -0.0020518845412880182...
2040    [-0.01595480740070343, -0.013429493643343449, ...
2041    [-0.037753768265247345, -0.017405441030859947,...
2042    [0.019308919087052345, 0.007406963966786861, 0...
2043    [-0.0045866332948207855, 0.001447434420697391,...
Name: embedding, Length: 2044, dtype: object

In [53]:
#search without redis
from openai.embeddings_utils import get_embedding, cosine_similarity

# search through the symptoms for the most similar topic
def search_symptoms(df, symptoms, n=5, pprint=True):
    symptoms_embedding = get_embedding(symptoms, engine=embedding_model)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, symptoms_embedding))
    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .combined.str.replace("Topic: ", "")
        .str.replace("Symptoms: ", "")
    )
    if pprint:
        for r in results:
            print(r[:200])
            print()
    return results

In [54]:
results = search_symptoms(df, "back pain", n=5)

Title: Back Pain Meta Desc: Lumbago is pain on the lower part of your back. Back pain is very common. Treatment varies from medicines to surgery depending on the cause and kind of pain. Also Called: L

Title: Back Injuries Meta Desc: Back injuries can affect your range of motion. Common back injuries include sprains and strains, herniated discs and fractured vertebrae. Also Called:  Full Summary: <p

Title: Spine Injuries and Disorders Meta Desc: Spine injuries and spine disorders such as infection, scoliosis, and herniated disk can cause pain and other issues. Read about problems of the spine. Al

Title: Tailbone Disorders Meta Desc: The tailbone, or coccyx, is at the bottom of your backbone, or spine. Learn about tailbone disorders and what problems cause pain in your coccyx.  Also Called: Coc

Title: Dolor de espalda Meta Desc: La lumbalgia es el dolor en la parte baja de la espalda.  El dolor de espalda es uno de los problemas médicos más comunes. Tratamientos, prevención aquí. Als

In [55]:
results = search_symptoms(df, "memory loss", n=7)

Title: Memory Meta Desc: Did you forget where you put your keys? It's normal to forget things, but it can be a sign of memory problems. Read more on memory and memory loss. Also Called:  Full Summary:

Title: Mild Cognitive Impairment Meta Desc: In mild cognitive impairment (MCI), you have more memory and thinking problems than others your age, but they don't interfere with your everyday life. Also 

Title: Dementia Meta Desc: People with dementia have serious problems with two or more brain functions, such as memory and language. Affects the ability to do normal activities. Also Called: Senility 

Title: Memoria Meta Desc: ¿No recuerda si cerró bien la puerta? Es normal olvidar cosas, pero si ocurre muy a menudo puede ser un signo de problemas de la memoria. Entérese aquí. Also Called:  Full Su

Title: Deterioro cognitivo leve Meta Desc: Las personas con deterioro cognitivo leve tienen más problemas de memoria que otras. Puede ser una señal de demencia. Entérese aquí. Also Called:  Fu

In [91]:
results = search_symptoms(df, "my ribs are hurting", n=10)

Title: Abdominal Pain Meta Desc: Stomach aches can be painful. Find out what might be the cause of your abdominal pain.  Also Called: Bellyache Full Summary: <p>Your abdomen extends from below your ch

Title: Back Injuries Meta Desc: Back injuries can affect your range of motion. Common back injuries include sprains and strains, herniated discs and fractured vertebrae. Also Called:  Full Summary: <p

Title: Chest Injuries and Disorders Meta Desc: Your chest is packed with vital organs, like the esophagus, lungs, and heart. Learn about the different types of chest injuries and chest disorders. Also

Title: Polymyalgia Rheumatica Meta Desc: Polymyalgia rheumatica (PMR) is a disorder that causes muscle pain and stiffness in your neck, shoulders and hips. Read about diagnosis and treatment. Also Cal

Title: Back Pain Meta Desc: Lumbago is pain on the lower part of your back. Back pain is very common. Treatment varies from medicines to surgery depending on the cause and kind of pain. Also C

In [92]:
results = search_symptoms(df, "my ribs are hurting, when i am running", n=10)

Title: Abdominal Pain Meta Desc: Stomach aches can be painful. Find out what might be the cause of your abdominal pain.  Also Called: Bellyache Full Summary: <p>Your abdomen extends from below your ch

Title: Back Injuries Meta Desc: Back injuries can affect your range of motion. Common back injuries include sprains and strains, herniated discs and fractured vertebrae. Also Called:  Full Summary: <p

Title: Thoracic Outlet Syndrome Meta Desc: In thoracic outlet syndrome (TOS), a compressed nerve or blood vessel causes pain in your arm, shoulder, and neck. Read about the types and treatment. Also C

Title: Sports Injuries Meta Desc: Get the facts on sports injuries, including the most common types, and read about the treatments available Also Called:  Full Summary: <p>Exercising is good for you, 

Title: Chest Pain Meta Desc: Chest pain can be caused by a heart attack, but it can have other causes. Learn about what they are, and when you should get immediate medical help. Also Called:  

In [ ]:
results = search_symptoms(df, "my ribs are hurting, when i am running, ", n=10)

In [57]:
# start redis using the docker-compose file in the same folder
! docker compose up -d

Container redis-vector-db-1  Running


In [83]:
# connect to redis
import redis
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField
)

REDIS_HOST =  "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = "" # default for passwordless Redis

# Connect to Redis
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD
)
redis_client.ping()

True

In [84]:
# Create a search index in Redis

# Constants
VECTOR_DIM = len(df['embedding'].values[0]) # length of the vectors
VECTOR_NUMBER = len(df)                 # initial number of vectors
INDEX_NAME = "embeddings-reviews-index"   # name of the search index
PREFIX = "doc"                            # prefix for the document keys
DISTANCE_METRIC = "COSINE"                # distance metric for the vectors (ex. COSINE, IP, L2)

In [60]:
# Define RedisSearch fields for each of the columns in the dataset, map the embedding column to a vector field and set the distance metric
topic = TextField(name="title", weight=1.0)
overview = TextField(name="meta-desc", weight=1.0)
symptoms = TextField(name="full-summary", weight=1.0)
url = TextField(name="url", weight=1.0)
embedding = VectorField("embedding",
                        "FLAT", {
                            "TYPE": "FLOAT32",
                            "DIM": VECTOR_DIM,
                            "DISTANCE_METRIC": DISTANCE_METRIC,
                            "INITIAL_CAP": VECTOR_NUMBER,
                        }
            )
fields = [topic, overview, symptoms, url, embedding]


In [61]:
# Check if index exists
try:
    redis_client.ft(INDEX_NAME).info()
    print("Index already exists")
except:
    # Create RediSearch Index
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
)

Index already exists


In [79]:
def index_documents(client: redis.Redis, prefix: str, documents: pd.DataFrame):
    records = documents.to_dict("records")
    offset = 1452 # number of documents already indexed
    for i, doc in enumerate(records):
        try:
            key = f"{prefix}:{offset + i}"

            # create byte vectors for title and content
            embedding = np.array(doc["embedding"], dtype=np.float32).tobytes()

            # replace list of floats with byte vectors
            doc["embedding"] = embedding

            client.hset(key, mapping = doc)
        except:
            print(f"Error indexing document {i}: {doc['title']}")

In [69]:
offset = 2000
i = 1
print(f"{PREFIX}:{offset + i}")

doc:2001


In [80]:
index_documents(redis_client, PREFIX, df)
print(f"Loaded {redis_client.info()['db0']['keys']} documents in Redis search index with name: {INDEX_NAME}")

Error indexing document 0: A1C
Error indexing document 1: Abdominal Pain
Error indexing document 2: Abortion
Error indexing document 3: Aborto
Error indexing document 4: Abscesos
Error indexing document 5: Abscess
Error indexing document 6: Abuso de las personas mayores
Error indexing document 7: Abuso sexual de menores
Error indexing document 8: Accidente cerebrovascular
Error indexing document 9: Accidente cerebrovascular hemorrágico
Error indexing document 10: Accidente cerebrovascular isquémico
Error indexing document 11: Acidez estomacal
Error indexing document 13: Acné
Error indexing document 15: Acupuncture
Error indexing document 16: Acupuntura
Error indexing document 17: Acute Bronchitis
Error indexing document 18: Acute Flaccid Myelitis
Error indexing document 22: Adenoides
Error indexing document 23: Adenoids
Error indexing document 24: Adherencias
Error indexing document 25: Adhesions
Error indexing document 26: Adicción a los juegos de azar
Error indexing document 27: Adre

In [82]:
df

,id,meta-desc,title,language,also-called,full-summary,group,language-mapped-topic,other-language,primary-institute,see-reference,related-topic,site,url,combined,n_tokens,embedding,similarity
0,6308,"If you are being tested for Type 2 diabetes, y...",A1C,English,Hemoglobin A1C test,"<p>A1C is a blood test for <a href=""https://me...",Diabetes Mellitus,Prueba de hemoglobina glicosilada (HbA1c),Spanish,National Institute of Diabetes and Digestive a...,Hemoglobin A1c,None,NaN,https://medlineplus.gov/a1c.html,Title: A1C Meta Desc: If you are being tested ...,437,"[-0.004037505481392145, -0.003933656960725784,...",0.704616
1,3061,Stomach aches can be painful. Find out what mi...,Abdominal Pain,English,Bellyache,<p>Your abdomen extends from below your chest ...,Symptoms,Dolor abdominal,Spanish,None,Stomach Ache,Pelvic Pain,NaN,https://medlineplus.gov/abdominalpain.html,Title: Abdominal Pain Meta Desc: Stomach aches...,254,"[0.003938998095691204, -0.005209034308791161, ...",0.734338
2,122,An abortion is a medical procedure to end a pr...,Abortion,English,Induced Abortion,<p>An abortion is a procedure to end a pregnan...,Female Reproductive System,Aborto,Vietnamese,None,None,None,NaN,https://medlineplus.gov/abortion.html,Title: Abortion Meta Desc: An abortion is a me...,195,"[-0.022983038797974586, 0.015163933858275414, ...",0.686476
3,2238,Un aborto es un procedimiento médico para term...,Aborto,Spanish,Interrupción del embarazo,<p>Un aborto es un procedimiento para terminar...,Sistema reproductor femenino,Abortion,None,None,None,None,NaN,https://medlineplus.gov/spanish/abortion.html,Title: Aborto Meta Desc: Un aborto es un proce...,253,"[-0.031313538551330566, 0.0003964322677347809,...",0.674296
4,3064,Los abscesos se llenan de pus y pueden aparece...,Abscesos,Spanish,None,<p>Un absceso es una cavidad donde se acumula ...,Infecciones,Abscess,None,Instituto Nacional de Alergias y Enfermedades ...,None,Enfermedades infecciosas,NaN,https://medlineplus.gov/spanish/abscess.html,Title: Abscesos Meta Desc: Los abscesos se lle...,318,"[0.0016349778743460774, -0.0014277726877480745...",0.713272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2039,6604,¿Qué cantidad de ejercicio se debe hacer? Aquí...,¿Cuánto ejercicio debo hacer?,Spanish,None,<p>Hacer ejercicio en forma regular es una de ...,Aptitud física y ejercicio,How Much Exercise Do I Need?,None,None,None,None,NaN,https://medlineplus.gov/spanish/howmuchexercis...,Title: ¿Cuánto ejercicio debo hacer? Meta Desc...,1162,"[-0.009836418554186821, -0.0020518845412880182...",0.693710
2040,1907,"Si está embarazada, es probable que haya escuc...",Ácido fólico,Spanish,Ácido pteroilglutámico,"<p>El ácido fólico es una <a href=""https://med...",Alimentos y nutrición,Folic Acid,None,Oficina de Suplementos Dietéticos,None,Vitaminas,NaN,https://medlineplus.gov/spanish/folicacid.html,Title: Ácido fólico Meta Desc: Si está embaraz...,375,"[-0.01595480740070343, -0.013429493643343449, ...",0.696712
2041,1749,El ántrax o carbunco es ocasionado por una bac...,Ántrax,Spanish,Carbunco,<p>El ántrax es una enfermedad causada por el ...,Infecciones,Anthrax,None,Instituto Nacional de Alergias y Enfermedades ...,Carbunco,Biodefensa y bioterrorismo,NaN,https://medlineplus.gov/spanish/anthrax.html,Title: Ántrax Meta Desc: El ántrax o carbunco ...,484,"[-0.037753768265247345, -0.017405441030859947,...",0.713275
2042,2108,Las úlceras por presión son áreas de piel lesi...,Úlcera por presión,Spanish,Llagas por presión,<p>Las úlceras por presión son áreas de piel l...,"Piel, cabello y uñas",Pressure Sores,None,Instituto Nacional de Artritis y Enfermedades ...,Úlcera por decúbito,None,NaN,https://medlineplus.gov/spanish/pressuresores....,Title: Úlcera por presión Meta Desc: Las úlcer...,329,"[0.019308919087052345, 0.007406963966786861, 0...",0.695559


In [89]:
# Run a search query and return the results
from typing import List
def search_redis(
        redis_client: redis.Redis,
        user_query: str,
        index_name: str = INDEX_NAME,
        vector_field: str = "embedding",
        return_fields: list = ["topic", "overview", "symptoms", "url", "vector_score"],
        hybrid_fields = "*",
        k: int = 20,
        print_results: bool = True,
) -> List[dict]:
    """
    Search Redis for a given query and return the results.
    :param redis_client: Redis client
    :param user_query: Query string
    :param index_name: Name of the index to search in
    :param vector_field: Name of the vector field
    :param return_fields: List of fields to return
    :param hybrid_fields: List of fields to use for hybrid search
    :param k: Number of results to return
    :param print_results: Whether to print the results
    :return: List of results
    """
    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(input=user_query,
                                             model="text-embedding-ada-002",
                                             )["data"][0]['embedding']
    
    # Prepare the query
    base_query = f'{hybrid_fields}=>[KNN {k} @{vector_field} $vector AS vector_score]'

    query = (
        Query(base_query)
        .return_fields(*return_fields)
        .sort_by("vector_score")
        .paging(0, k)
        .dialect(2)
    )

    params_dict = {
        "vector": np.array(embedded_query).astype(dtype=np.float32).tobytes()
    }

    # perforrm vector search
    results = redis_client.ft(index_name).search(query, params_dict)

    # Print the results
    if print_results:
        for i, result in enumerate(results.docs):
            print(f"Rank: {i}")
            print(f"Topic: {result.topic}")
            print(f"Overview: {result.overview}")
            print(f"Symptoms: {result.symptoms}")
            print(f"URL: {result.url}")
            score = 1 - float(result.vector_score)
            print(f"Score: {round(score, 3)})")
            print()

    return results.docs

In [90]:
# Search redis
results = search_redis(redis_client, 'back pain', k=10)

Rank: 0
Topic: Lumbar Pain
Overview: We often bring on our back problems through bad habits, such as: The spine is actually a stack of 24 bones called vertebrae. A healthy spine is S-shaped when viewed from the side. It curves back at your shoulders and inward at your neck and small of your back. It houses and protects your spinal cord, the network of nerves that transmit feeling and control movement throughout your entire body.  One of the more common types of back pain comes from straining the bands of muscles surrounding the spine. It happens most often in the curve of the low back and the base of the neck. These areas support more weight than your upper and mid back, which are less prone to trouble. Injuries from contact sports, accidents, and falls can cause problems ranging from minor muscle strains, to herniated disks, to fractures that damage the spinal column or cord. Stabbing low back pain could be from muscle spasms, when your muscles seize up and don't relax, like a cramp. 

AttributeError: 'Document' object has no attribute 'topic'